In [102]:
include("asinclude.jl")

In [103]:
## Demo

asinclude.@asinclude("mymodule", quote
    import Base.start
    import Base.next
    import Base.done

    export MyType, IterType, bar

    type MyType
        x
        baz::Int
    end

    bar(x) = 2x
    show(io, a::MyType) = print(io, "MyType $(a.baz)")
    
    type IterType
    baz
    end

    function start(c::IterType)
        N = c.baz
        state = 0
        return state
    end

    function next(c::IterType, state)
        return state + 1, state + 1
    end

    function done(c::IterType, state)
        if isempty(state)
            return true
        end
        state += 1
        i = 1
        if state > c.baz
            return true
        end
        return false
    end
end)

println(mymodule.MyType(2, 5))
for i = mymodule.IterType(5)
    println(i)
end

Embedding exported names
MyType(2,5)
1
2
3
4
5


In [104]:
asinclude.@asinclude("mymodule2", quote
    import Base: start, done, next
    export MyType, IterType, bar

    type MyType
        baz::Int
    end

    bar(x) = 2x
    show(io, a::MyType) = print(io, "MyType $(a.baz)")
    
    type IterType
        baz
    end

    function start(c::IterType)
        N = c.baz
        state = 0
        return state
    end

    function next(c::IterType, state)
        return state + 1, state + 1
    end

    function done(c::IterType, state)
        if isempty(state)
            return true
        end
        state += 1
        i = 1
        if state > c.baz
            return true
        end
        return false
    end
end)

println(mymodule.MyType(10))
for i = mymodule.IterType(5)
    println(i)
end

Embedding exported names


LoadError: `MyType` has no method matching MyType(::Int64)
while loading In[104], in expression starting on line 39

In [75]:
macro dynamic_import(modules)
           (modules = eval(modules))::Vector{Symbol}
           ex = Expr(:toplevel)
           for m in modules
               push!(ex.args, Expr(:import, m))
           end
           return ex
       end

In [81]:
eval(Expr(:import, symbol("mymodule")))

In [85]:
names(eval(symbol("mymodule")))

4-element Array{Symbol,1}:
 :IterType
 :MyType  
 :mymodule
 :bar     

In [127]:
toks = split(string(quote
    import Base: start, next
    end), ':')[3:end]
specialform_handler["toplevel"](toks...)

toplevel, 
($(Expr(


LoadError: key not found: "toplevel, "
while loading In[127], in expression starting on line 4

In [126]:
specialform_handler = {
    "import" => (args...) -> begin
    args = map(x -> replace(x, r",|\s|\)|(#.*$)", ""), args)
    args = filter(x->length(x)>0, [args...])
    return (args[1] * " " * join(args[2:end], '.'))
end,
    "export" => (args...) -> begin
    args = map(x -> replace(x, r",|\s|\)|(#.*$)", ""), args)
    args = filter(x->length(x)>0, [args...])
    return (args[1] * " " * join(args[2:end], ','))
end,
    "toplevel" => (args...) -> begin
    args = map(x -> replace(x, r",|\s|\)|(#.*$)", ""), args)
    entries = String[]
    currententry = String[]
    for token in args
        println(token)
        if contains(token, "\$")
            push!(entries, specialform_handler[currententry[1]](currententry...))
            currententry = []
        end
        push!(currententry, token)
    end
    return join(map(x -> specialform_handler[x[1]](x), entries), "\n")
end
}

Dict{Any,Any} with 3 entries:
  "export"   => (anonymous function)
  "toplevel" => (anonymous function)
  "import"   => (anonymous function)

In [34]:
for n in [:data]
    println(n)
end

data


In [125]:
aas = Array(String, 2)
push!(aas, ["hello";])

LoadError: `convert` has no method matching convert(::Type{String}, ::Array{ASCIIString,1})
while loading In[125], in expression starting on line 2

In [9]:
∈([1,2,3], [4])

false